# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# defeine the file path
cities_path = '../WeatherPy/output_data/cities.csv'

# read the data from weather module and check 
cities_df = pd.read_csv(cities_path)
cities_df.dropna(inplace = True)
cities_df.head()

,City,Lat,Lng,Country,Max Temp (C),Humidity,Cloudiness,Wind Speed (Kmph),Date
0,Puerto Escondido,15.8500,-97.0667,MX,24.46,64,40,0.00,1624192490
1,Punta Arenas,-53.1500,-70.9167,CL,3.06,81,0,3.60,1624192491
2,Hearst,49.6835,-83.6665,CA,14.14,77,100,2.35,1624192491
3,Rikitea,-23.1203,-134.9692,PF,23.82,77,87,8.93,1624192491
4,Ushuaia,-54.8000,-68.3000,AR,1.81,80,20,2.06,1624192491


In [3]:
# check for number of cities read
len(cities_df)

567

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [5]:
# define humidity as the weight for the heat map
humidity = cities_df['Humidity']

# set loaction from lat and lng
locations = cities_df[["Lat", "Lng"]]

In [6]:
# define layout and plot heatmap
figure_layout = {
    'width': '950px',
    'height': '480px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)

# Add layer to the map
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='480px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# filter cities with 
# Max temperature between 20C and 26C
# Wind speed less than 15 kmph
# Cloudiness less than 1
cities_df = cities_df[(cities_df['Max Temp (C)'] >= 20) & (cities_df['Max Temp (C)'] <= 26) & 
                          (cities_df['Wind Speed (Kmph)'] < 15) & (cities_df['Cloudiness'] < 1)]  

# drop rows with null values
cities_df.dropna(inplace = True)
cities_df.head()

,City,Lat,Lng,Country,Max Temp (C),Humidity,Cloudiness,Wind Speed (Kmph),Date
64,Kunashak,55.7032,61.5498,RU,23.29,42,0,9.40,1624192507
105,Arraial do Cabo,-22.9661,-42.0278,BR,23.98,69,0,4.63,1624192518
124,Diamantino,-14.4086,-56.4461,BR,21.76,55,0,2.61,1624192523
390,Vila Franca do Campo,37.7167,-25.4333,PT,22.23,62,0,4.84,1624192593
501,Khani,41.9563,42.9566,GE,24.76,33,0,1.50,1624192621


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Copy cledat data set and add two new columns
hotel_df = cities_df

hotel_df['Hotel Name'] = ""
hotel_df['Rating'] = ""
hotel_df.head()

,City,Lat,Lng,Country,Max Temp (C),Humidity,Cloudiness,Wind Speed (Kmph),Date,Hotel Name,Rating
64,Kunashak,55.7032,61.5498,RU,23.29,42,0,9.40,1624192507,,
105,Arraial do Cabo,-22.9661,-42.0278,BR,23.98,69,0,4.63,1624192518,,
124,Diamantino,-14.4086,-56.4461,BR,21.76,55,0,2.61,1624192523,,
390,Vila Franca do Campo,37.7167,-25.4333,PT,22.23,62,0,4.84,1624192593,,
501,Khani,41.9563,42.9566,GE,24.76,33,0,1.50,1624192621,,


In [9]:
# find the closest hotel(lodging) within 5000 meter radius of the city
target_radius = 5000
target_type = "lodging"

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": target_radius,
    "type": target_type,
    "key": g_key,
}

# use iterrows to iterate through hotel dataframe
for index, row in hotel_df.iterrows():

    # get lat and lng of the city
    city_loc = str(row['Lat']) + ', ' + str(row['Lng'])
    
    # add keyword to params dict ie the city name
    params['location'] = city_loc

    # create url and make API request
    response = requests.get(base_url, params=params).json()
    
    # get results
    results = response['results']
    
    try:
        # get hotel name and it's lat & lng and rating
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Lat'] = results[0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Lng'] = results[0]['geometry']['location']['lng']
        hotel_df.loc[index, 'Rating'] = results[0]['rating']        
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [11]:
# define humidity as the weight for the heat map
humidity = hotel_df['Humidity']

# create heat, maker and symbol layers
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)
marker_layer = gmaps.marker_layer(locations)
symbol_layer = gmaps.symbol_layer(locations, info_box_content=hotel_info)

# define layout and plot heatmap
figure_layout = {
    'width': '950px',
    'height': '480px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Add the layers
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig.add_layer(symbol_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='480px', margin='0 auto 0 auto', padding='1px', wi…